<a href="https://colab.research.google.com/github/hieunguyen-cyber/AI_Project_CBH_GHHKM/blob/main/Predicting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sử dụng mô hình YOLOv10 đã được huấn luyện để nhận diện biên báo giao thông**


Ở đây, chúng ta sẽ sử dụng file trọng số **best.pt** thu được trong giai đoạn huấn luyện để áp dụng vào mô hình YOLOv10 nhằm giải quyết bài toán nhận diện biển báo giao thông.

**Chú ý**: Kết nối với GPU trước khi sử dụng

Nhóm thực hiện: CBH_GHHKM

---

###**Bước 1**: Tải file trọng số **best.pt** lên Google Colab

In [1]:
!git clone https://github.com/hieunguyen-cyber/Weight_AIBK_2024.git
!mv /content/Weight_AIBK_2024/best.pt /content

Cloning into 'Weight_AIBK_2024'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 14.88 MiB | 10.16 MiB/s, done.


###**Bước 2**: Tải dữ liệu muốn dự đoán

Để các ảnh cần dự đoán trong 1 folder tên **test_img** trong Google Drive

Có thể tham khảo bộ ảnh sau: [test_img](https://l.messenger.com/l.php?u=https%3A%2F%2Fdrive.google.com%2Fdrive%2Ffolders%2F1totO1tw2wZ_Fv2RSG72_4tuUZC66LdpY%3Fusp%3Ddrive_link&h=AT1vv7_RcZ3qzSh9Wt_YqVTeYw9jVWh1ibVWN_Egp3bO1hhzk_8POVws0soYLXyYsVMgxOIMmyJxYB26wukr3E_dMXS6biCLG34T6ZjkYm-Fmt2djZGnzr4eHfexZl-rgLFb1g)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###**Bước 3**: Cài đặt YOLOv10

In [ ]:
!git clone https://github.com/THU-MIG/yolov10

###**Bước 4**: Tiến hành dự đoán

In [ ]:
%cd /content/yolov10

from ultralytics import YOLOv10
import os

MODEL_PATH = '/content/best.pt'
model = YOLOv10(MODEL_PATH)

input_folder = '/content/drive/MyDrive/test_img'
output_folder = '/content/output_images'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        input_path = os.path.join(input_folder, filename)

        results = model.predict(input_path)

        output_path = os.path.join(output_folder, filename)
        results[0].save(output_path)

        print(f'Processed and saved: {output_path}')


* Ảnh đã dự đoán được lưu tại folder **output_images**